In [1]:
### 常规分析包
import os
import time
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import norm, pearsonr, spearmanr
from scipy.spatial import distance
### 绘图包
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import seaborn as sns
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42 # 输出PDF文字可编辑
### 其他
import cloudpickle as pickle
import warnings
warnings.filterwarnings("ignore") # 报错提示
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' #last_expr

In [2]:
meta = pd.read_csv('../Data_preprocess/1.1.MetaData.tsv', sep='\t', index_col=0)
meta.head(2)
#data = pd.read_csv('../Data/16S/MZJ_dada2_table_ASV.tsv', sep='\t', index_col=0, skiprows=1)
data = pd.read_csv('../Data/16S/DADA2_table/MZJ_dada2_table_L6.tsv', sep='\t', index_col=0, skiprows=1)
data.columns = [i if i!='11_10' else 'C11_10' for i in data.columns]

data.shape
data.head(2)

,SampleID,SampleID_1,Child_ID,Sample_type,Day,Time,MotherID,WMS,Gender,Mode of delivery,...,Day of the last sample,Number of samples,Day of introducing solid food,Day of breastfeeding cessation,Solid food,Breastfeeding,"Feeding mode(BFD:0,MFD:1)","Sibling(Y:1,N:0)","PAE(Y:1,N:0)","Allergy(Y:1,N:0)"
ID,,,,,,,,,,,,,,,,,,,,,
C11_10,11_10,D_10,11,C,25.0,T10,M11_M,0,Female,C-section,...,352.0,25.0,176.0,356.0,0.0,1.0,0.0,0.0,0.0,1.0
C11_11,11_11,D_11,11,C,29.0,T11,M11_M,0,Female,C-section,...,352.0,25.0,176.0,356.0,0.0,1.0,0.0,0.0,0.0,1.0


(511, 466)

,C11_10,C1_11,C1_12,C1_13,C1_14,C1_15,C1_16,C1_2,C1_21,C1_22,...,M21_M,M25_M,M26_M,M28_M,M29_M,M31_M,M32_M,M34_M,M35_M,M4_M
#OTU ID,,,,,,,,,,,,,,,,,,,,,
k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacteriales;f__Enterobacteriaceae;__,10611.0,247.0,699.0,1819.0,421.0,194.0,2550.0,9995.0,7853.0,1220.0,...,0.0,0.0,0.0,0.0,0.0,23.0,4733.0,16.0,116.0,99.0
k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Peptostreptococcaceae;__,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
path = '../Results_0609/6.4. Sparcc_raw/'
r_threshold = 0.15
p_threshold = 0.05
def find_phylum(taxa):
    phylums = ['Proteobacteria', 'Firmicutes', 'Bacteroidetes', 'Actinobacteria']
    for p in phylums:
        if p in taxa:
            return p
    return 'Others'

all_nodes = set()

for t in range(2, 28):
    sparcc = pd.read_csv(path+'SparCC_T'+str(t)+'.tsv', sep='\t', index_col=0)
    #sparcc.head(1)
    net = sparcc.loc[(sparcc['cor'].abs()>=r_threshold)&(sparcc['p']<=p_threshold), :]
    t, len(set(sparcc['t1'])), len(set(sparcc['t2'])), net.shape[0]
    nodes_set = set()
    links_set = set()
    nodes = []
    links = []
    for i in net.index:
        t1 = net.loc[i, 't1']
        t2 = net.loc[i, 't2']
        r = net.loc[i, 'cor']
        p = net.loc[i, 'p']
        if t1 not in nodes_set:
            nodes_set.add(t1)
            phylum = find_phylum(t1)
            nodes.append([t1, t1.split(';')[-1], phylum])
            all_nodes.add((t1, t1.split(';')[-1], phylum))
        if t2 not in nodes_set:
            nodes_set.add(t2)
            phylum = find_phylum(t2)
            nodes.append([t2, t2.split(';')[-1], phylum])
            all_nodes.add((t2, t2.split(';')[-1], phylum))
        if tuple(sorted([t1, t2])) not in links_set:
            links_set.add(tuple(sorted([t1, t2])))
            links.append([t1, t2, abs(r), r, p, np.sign(r)])
    nodes = pd.DataFrame(nodes, columns=['Id', 'Label', 'Phylum'])
    links = pd.DataFrame(links, columns=['Source', 'Target', 'Weight', 'r', 'p', 'signr'])
    # save
    nodes.to_csv(path+'SparCC_T'+str(t)+'_nodes('+str(r_threshold)+').csv')
    links.to_csv(path+'SparCC_T'+str(t)+'_links('+str(r_threshold)+').csv')

(2, 263, 263, 1030)

(3, 168, 168, 32)

(4, 123, 123, 152)

(5, 87, 87, 80)

(6, 145, 145, 548)

...

(23, 118, 118, 435)

(24, 201, 201, 115)

(25, 73, 73, 90)

(26, 101, 101, 174)

(27, 84, 84, 124)

In [14]:
all_nodes = pd.DataFrame(all_nodes, columns=['Id', 'Label', 'Phylum'])
all_nodes.to_csv(path+'SparCC_All_nodes('+str(r_threshold)+').csv')

In [16]:
sparcc = pd.read_csv(path+'SparCC_Child.tsv', sep='\t', index_col=0)
#sparcc.head(1)
net = sparcc.loc[(sparcc['cor'].abs()>=r_threshold)&(sparcc['p']<=p_threshold), :]
t, len(set(sparcc['t1'])), len(set(sparcc['t2'])), net.shape[0]
nodes_set = set()
links_set = set()
nodes = []
links = []
for i in net.index:
    t1 = net.loc[i, 't1']
    t2 = net.loc[i, 't2']
    r = net.loc[i, 'cor']
    p = net.loc[i, 'p']
    if t1 not in nodes_set:
        nodes_set.add(t1)
        phylum = find_phylum(t1)
        nodes.append([t1, t1.split(';')[-1], phylum])
        all_nodes.add((t1, t1.split(';')[-1], phylum))
    if t2 not in nodes_set:
        nodes_set.add(t2)
        phylum = find_phylum(t2)
        nodes.append([t2, t2.split(';')[-1], phylum])
        all_nodes.add((t2, t2.split(';')[-1], phylum))
    if tuple(sorted([t1, t2])) not in links_set:
        links_set.add(tuple(sorted([t1, t2])))
        links.append([t1, t2, abs(r), r, p, np.sign(r)])
nodes = pd.DataFrame(nodes, columns=['Id', 'Label', 'Phylum'])
links = pd.DataFrame(links, columns=['Source', 'Target', 'Weight', 'r', 'p', 'signr'])
# save
nodes.to_csv(path+'SparCC_Child_nodes('+str(r_threshold)+').csv')
links.to_csv(path+'SparCC_Child_links('+str(r_threshold)+').csv')

(27, 221, 221, 689)

,Id,Label,Phylum
0,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,g__Robinsoniella__,FirmicutesProteobacteria
1,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Kaistobacter__,ProteobacteriaProteobacteria
2,k__Bacteria;p__Planctomycetes;c__[Brocadiae];o...,____,OthersProteobacteria
3,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,g__Vibrio__,ProteobacteriaProteobacteria
4,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...,g__Bacteroides__,BacteroidetesProteobacteria
...,...,...,...
405,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g____,ProteobacteriaProteobacteria
406,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,____,FirmicutesProteobacteria
407,k__Bacteria;p__Chlorobi;c__OPB56;o__;f__;g__k_...,g____,OthersProteobacteria
408,k__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,____,OthersProteobacteria


,Id,Label,Phylum
0,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,g__Robinsoniellag__Bacteroides,FirmicutesBacteroidetes
1,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Kaistobacterg__Bacteroides,ProteobacteriaBacteroidetes
2,k__Bacteria;p__Planctomycetes;c__[Brocadiae];o...,__g__Bacteroides,OthersBacteroidetes
3,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,g__Vibriog__Bacteroides,ProteobacteriaBacteroidetes
4,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...,g__Bacteroidesg__Bacteroides,BacteroidetesBacteroidetes
...,...,...,...
405,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__g__Bacteroides,ProteobacteriaBacteroidetes
406,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,__g__Bacteroides,FirmicutesBacteroidetes
407,k__Bacteria;p__Chlorobi;c__OPB56;o__;f__;g__k_...,g__g__Bacteroides,OthersBacteroidetes
408,k__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,__g__Bacteroides,OthersBacteroidetes


,Id,Label,Phylum
0,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,g__Robinsoniellag__Streptococcus,FirmicutesFirmicutes
1,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Kaistobacterg__Streptococcus,ProteobacteriaFirmicutes
2,k__Bacteria;p__Planctomycetes;c__[Brocadiae];o...,__g__Streptococcus,OthersFirmicutes
3,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,g__Vibriog__Streptococcus,ProteobacteriaFirmicutes
4,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...,g__Bacteroidesg__Streptococcus,BacteroidetesFirmicutes
...,...,...,...
405,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__g__Streptococcus,ProteobacteriaFirmicutes
406,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,__g__Streptococcus,FirmicutesFirmicutes
407,k__Bacteria;p__Chlorobi;c__OPB56;o__;f__;g__k_...,g__g__Streptococcus,OthersFirmicutes
408,k__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,__g__Streptococcus,OthersFirmicutes


,Id,Label,Phylum
0,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,g__Robinsoniellag__Veillonella,FirmicutesFirmicutes
1,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Kaistobacterg__Veillonella,ProteobacteriaFirmicutes
2,k__Bacteria;p__Planctomycetes;c__[Brocadiae];o...,__g__Veillonella,OthersFirmicutes
3,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,g__Vibriog__Veillonella,ProteobacteriaFirmicutes
4,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...,g__Bacteroidesg__Veillonella,BacteroidetesFirmicutes
...,...,...,...
405,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__g__Veillonella,ProteobacteriaFirmicutes
406,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,__g__Veillonella,FirmicutesFirmicutes
407,k__Bacteria;p__Chlorobi;c__OPB56;o__;f__;g__k_...,g__g__Veillonella,OthersFirmicutes
408,k__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,__g__Veillonella,OthersFirmicutes


...

,Id,Label,Phylum
0,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,g__Robinsoniellag__Campylobacter,FirmicutesProteobacteria
1,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Kaistobacterg__Campylobacter,ProteobacteriaProteobacteria
2,k__Bacteria;p__Planctomycetes;c__[Brocadiae];o...,__g__Campylobacter,OthersProteobacteria
3,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,g__Vibriog__Campylobacter,ProteobacteriaProteobacteria
4,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...,g__Bacteroidesg__Campylobacter,BacteroidetesProteobacteria
...,...,...,...
405,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__g__Campylobacter,ProteobacteriaProteobacteria
406,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,__g__Campylobacter,FirmicutesProteobacteria
407,k__Bacteria;p__Chlorobi;c__OPB56;o__;f__;g__k_...,g__g__Campylobacter,OthersProteobacteria
408,k__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,__g__Campylobacter,OthersProteobacteria


,Id,Label,Phylum
0,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,g__Robinsoniellag__Flavobacterium,FirmicutesBacteroidetes
1,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Kaistobacterg__Flavobacterium,ProteobacteriaBacteroidetes
2,k__Bacteria;p__Planctomycetes;c__[Brocadiae];o...,__g__Flavobacterium,OthersBacteroidetes
3,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,g__Vibriog__Flavobacterium,ProteobacteriaBacteroidetes
4,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...,g__Bacteroidesg__Flavobacterium,BacteroidetesBacteroidetes
...,...,...,...
405,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__g__Flavobacterium,ProteobacteriaBacteroidetes
406,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,__g__Flavobacterium,FirmicutesBacteroidetes
407,k__Bacteria;p__Chlorobi;c__OPB56;o__;f__;g__k_...,g__g__Flavobacterium,OthersBacteroidetes
408,k__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,__g__Flavobacterium,OthersBacteroidetes


,Id,Label,Phylum
0,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,g__Robinsoniellag__,FirmicutesOthers
1,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Kaistobacterg__,ProteobacteriaOthers
2,k__Bacteria;p__Planctomycetes;c__[Brocadiae];o...,__g__,OthersOthers
3,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,g__Vibriog__,ProteobacteriaOthers
4,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...,g__Bacteroidesg__,BacteroidetesOthers
...,...,...,...
405,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__g__,ProteobacteriaOthers
406,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,__g__,FirmicutesOthers
407,k__Bacteria;p__Chlorobi;c__OPB56;o__;f__;g__k_...,g__g__,OthersOthers
408,k__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,__g__,OthersOthers


,Id,Label,Phylum
0,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,g__Robinsoniellag__Finegoldia,FirmicutesFirmicutes
1,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Kaistobacterg__Finegoldia,ProteobacteriaFirmicutes
2,k__Bacteria;p__Planctomycetes;c__[Brocadiae];o...,__g__Finegoldia,OthersFirmicutes
3,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,g__Vibriog__Finegoldia,ProteobacteriaFirmicutes
4,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...,g__Bacteroidesg__Finegoldia,BacteroidetesFirmicutes
...,...,...,...
405,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__g__Finegoldia,ProteobacteriaFirmicutes
406,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,__g__Finegoldia,FirmicutesFirmicutes
407,k__Bacteria;p__Chlorobi;c__OPB56;o__;f__;g__k_...,g__g__Finegoldia,OthersFirmicutes
408,k__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,__g__Finegoldia,OthersFirmicutes


,Id,Label,Phylum
0,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,g__Robinsoniellag__,FirmicutesOthers
1,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Kaistobacterg__,ProteobacteriaOthers
2,k__Bacteria;p__Planctomycetes;c__[Brocadiae];o...,__g__,OthersOthers
3,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...,g__Vibriog__,ProteobacteriaOthers
4,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...,g__Bacteroidesg__,BacteroidetesOthers
...,...,...,...
405,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__g__,ProteobacteriaOthers
406,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...,__g__,FirmicutesOthers
407,k__Bacteria;p__Chlorobi;c__OPB56;o__;f__;g__k_...,g__g__,OthersOthers
408,k__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,__g__,OthersOthers
